In [50]:
import pandas as pd
import json
import boto3
import awswrangler as wr
from tool import find_unique_combinations

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

boto3.setup_default_session(profile_name='vtn')

In [58]:
onw20240912 = pd.read_csv('03899361-57ee-473d-92b1-fd774f942178.csv', delimiter=',', header=0)
onw20240912

,journal_id,journal_date,journal_line,ledger,account,deptid,product,class_fld,chartfield1,chartfield2,chartfield3,project_id,monetary_amount,movement_flag,line_descr,jrnl_line_source,site,updated_at,acckey
0,88747,2023-01-31 00:00:00.000,2182,BUDGET,506007,110083.0,NaN,NaN,NaN,NaN,NaN,NaN,-9329.00,N,Overtime,NVS,vtn,1731858722,NaN
1,84676,2023-09-30 00:00:00.000,896,BUDGET,484102,110030.0,IPD,AR,2SPI,2SPI,2SPI,NaN,-229958.42,N,DISC GRAND SINGLE ROOM CHARGE,NVS,vtn,1731858722,2SPI-AR-2SPI-IPD
2,84668,2023-03-31 00:00:00.000,2340,BUDGET,420003,110040.0,IPD,TH,2WHC,2WHC,2WHC,NaN,-1430111.55,N,DOCTOR PROCEDURE CHARGE,NVS,vtn,1731858722,2WHC-TH-2WHC-IPD
3,84674,2023-08-31 00:00:00.000,2334,BUDGET,485501,110040.0,OPD,TH,2WHC,2WHC,2WHC,NaN,493.19,N,DISC FR NURSE SERVICE CHARGE,NVS,vtn,1731858722,2WHC-TH-2WHC-OPD
4,88716,2023-10-31 00:00:00.000,2888,BUDGET,485104,110020.0,IPD,AR,2ORT,2ORT,2ORT,NaN,1697.45,N,DISC FR MRI CHARGE,NVS,vtn,1731858722,2ORT-AR-2ORT-IPD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,88714,2023-08-31 00:00:00.000,2901,BUDGET,485505,110020.0,IPD,AR,2ORT,2ORT,2ORT,NaN,16.73,N,DISC FR NUTRITION SERVICE CHAR,NVS,vtn,1731858722,2ORT-AR-2ORT-IPD
499996,86898,2023-06-30 00:00:00.000,2725,BUDGET,506009,213010.0,NaN,NaN,NaN,NaN,NaN,NaN,2900.00,N,Incentive,NVS,vtn,1731858722,NaN
499997,84678,2023-11-30 00:00:00.000,3900,BUDGET,482005,120800.0,OPD,AR,2SLC,2SLC,2SLC,NaN,-71.66,N,DISCOUNT FR SLCC CHARGE,NVS,vtn,1731858722,2SLC-AR-2SLC-OPD
499998,88751,2023-04-30 00:00:00.000,3420,BUDGET,511002,402000.0,NaN,NaN,NaN,NaN,NaN,NaN,4105.40,N,Depre - Building Improvement,NVS,vtn,1731858722,NaN


In [59]:
onw20240912.columns

Index(['journal_id', 'journal_date', 'journal_line', 'ledger', 'account',
       'deptid', 'product', 'class_fld', 'chartfield1', 'chartfield2',
       'chartfield3', 'project_id', 'monetary_amount', 'movement_flag',
       'line_descr', 'jrnl_line_source', 'site', 'updated_at', 'acckey'],
      dtype='object')

In [60]:
onw20240912

,journal_id,journal_date,journal_line,ledger,account,deptid,product,class_fld,chartfield1,chartfield2,chartfield3,project_id,monetary_amount,movement_flag,line_descr,jrnl_line_source,site,updated_at,acckey
0,88747,2023-01-31 00:00:00.000,2182,BUDGET,506007,110083.0,NaN,NaN,NaN,NaN,NaN,NaN,-9329.00,N,Overtime,NVS,vtn,1731858722,NaN
1,84676,2023-09-30 00:00:00.000,896,BUDGET,484102,110030.0,IPD,AR,2SPI,2SPI,2SPI,NaN,-229958.42,N,DISC GRAND SINGLE ROOM CHARGE,NVS,vtn,1731858722,2SPI-AR-2SPI-IPD
2,84668,2023-03-31 00:00:00.000,2340,BUDGET,420003,110040.0,IPD,TH,2WHC,2WHC,2WHC,NaN,-1430111.55,N,DOCTOR PROCEDURE CHARGE,NVS,vtn,1731858722,2WHC-TH-2WHC-IPD
3,84674,2023-08-31 00:00:00.000,2334,BUDGET,485501,110040.0,OPD,TH,2WHC,2WHC,2WHC,NaN,493.19,N,DISC FR NURSE SERVICE CHARGE,NVS,vtn,1731858722,2WHC-TH-2WHC-OPD
4,88716,2023-10-31 00:00:00.000,2888,BUDGET,485104,110020.0,IPD,AR,2ORT,2ORT,2ORT,NaN,1697.45,N,DISC FR MRI CHARGE,NVS,vtn,1731858722,2ORT-AR-2ORT-IPD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,88714,2023-08-31 00:00:00.000,2901,BUDGET,485505,110020.0,IPD,AR,2ORT,2ORT,2ORT,NaN,16.73,N,DISC FR NUTRITION SERVICE CHAR,NVS,vtn,1731858722,2ORT-AR-2ORT-IPD
499996,86898,2023-06-30 00:00:00.000,2725,BUDGET,506009,213010.0,NaN,NaN,NaN,NaN,NaN,NaN,2900.00,N,Incentive,NVS,vtn,1731858722,NaN
499997,84678,2023-11-30 00:00:00.000,3900,BUDGET,482005,120800.0,OPD,AR,2SLC,2SLC,2SLC,NaN,-71.66,N,DISCOUNT FR SLCC CHARGE,NVS,vtn,1731858722,2SLC-AR-2SLC-OPD
499998,88751,2023-04-30 00:00:00.000,3420,BUDGET,511002,402000.0,NaN,NaN,NaN,NaN,NaN,NaN,4105.40,N,Depre - Building Improvement,NVS,vtn,1731858722,NaN


In [75]:
# onw20240912.apply(lambda x: len(x.unique()))

onw20240912[[
    'journal_id', 
    # 'journal_date', 
    'journal_line', 
    # 'ledger', 
    'account',
    # 'deptid', 
    # 'product', 
    # 'class_fld', 'chartfield1', 'chartfield2',
    # 'chartfield3', 
    # 'project_id', 
    'monetary_amount', 'movement_flag',
    'line_descr', 'jrnl_line_source', 'site', 'updated_at', 
    'acckey'

]].value_counts()
# onw20240912
# from itertools import combinations



journal_id    journal_line  account  monetary_amount  movement_flag  line_descr                      jrnl_line_source  site  updated_at  acckey          
84666         1             400001   -2054501.70      N              DRUG SOLD TO PATIENT  ORIGINAL  NVS               vtn   1731858722  2GI-TH-2GI-OPD      1
DFT2300011    31012         501107    300.00          N              Film Reported Fee               GOT               vtn   1731858728  2CAC-TH-2W7A-IPD    1
              31071         501101    1000.00         N              Consultant DF-Doctor Evaluate   GOT               vtn   1731858728  2CAC-QT-2CAC-OPD    1
              31070         501101    2200.00         N              Consultant DF-Doctor Evaluate   GOT               vtn   1731858728  2TJR-TH-2TJR-OPD    1
              31069         501101    1000.00         N              Consultant DF-Doctor Evaluate   GOT               vtn   1731858728  2TJR-AR-2TJR-OPD    1
                                                   

In [ ]:
def find_unique_combinations(df, cols):
    for r in range(1, len(cols) + 1):
        for combo in combinations(cols, r):
            # print(f"Checking combination: {combo}")

            # Create a combined key based on selected columns
            combined_key = df[list(combo)].apply(tuple, axis=1)
            print(list(combo))
            print(combined_key.nunique())

            # Check if the combination of columns is unique
            if combined_key.nunique() == len(df):
                print(f"Unique combination: {combo}")
                
    print("End of search")
    
find_unique_combinations(onw20240912, [
    'journal_id', 
    'journal_date', 
    'journal_line', 
    'ledger', 
    'account',
    'deptid', 
    'product', 
    'class_fld', 'chartfield1', 'chartfield2',
    'chartfield3', 
    'site'
    'project_id',
    'line_descr',
    'acckey'
])


['journal_id']
610
['journal_date']
54
['journal_line']
43145
['ledger']
2
['account']
383
['deptid']
137
['project_id']
795
['line_descr']
1558
['acckey']
2372
['journal_id', 'journal_date']
679
['journal_id', 'journal_line']
415389
['journal_id', 'ledger']
610
['journal_id', 'account']
13837
['journal_id', 'deptid']
10524
['journal_id', 'project_id']
1489
['journal_id', 'line_descr']
9597
['journal_id', 'acckey']
15096
['journal_date', 'journal_line']
242699
['journal_date', 'ledger']
55
['journal_date', 'account']
10139
['journal_date', 'deptid']
4888
['journal_date', 'project_id']
849
['journal_date', 'line_descr']
5612
['journal_date', 'acckey']
13260
['journal_line', 'ledger']
64469
['journal_line', 'account']
197906
['journal_line', 'deptid']
146926
['journal_line', 'project_id']
44435
['journal_line', 'line_descr']
169815
['journal_line', 'acckey']
169531
['ledger', 'account']
668
['ledger', 'deptid']
265
['ledger', 'project_id']
796
['ledger', 'line_descr']
1716
['ledger', 'ac

In [6]:
onw20240912[onw20240912['OrderItemID and Accession No']==13652444].sort_values('OrderItemSubID and Accession No')

,BU,en,OrderDate,Billing Sub Group,Item code,Description,Package Code,hn,Name,Account Code,Discoun Account,Department,First Loc,Patient Loc,Rec Loc,Sub Type,nation,curr,Amount Before,Discount,Bill Amount,Doctor Group,Doctor Type,Doctor Code,Doctor Name,Qty,OrderItemID and Accession No,OrderItemSubID and Accession No,Episode Id,Patient Id
9457,VTN01,I23-009855,19/12/2023,11010001,MLXHYDR01,Hydrocortisone cream 1% (L) (5,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400002,480002,230300,12REH,12REH,1PHI,IPD,AR,THB,0.00,0.00,0.02,93231.0,DRMS01,93231,พีรวุฒิ ตันติสุวณิชย์กุล นพ.,300.0,13652444,34,8486182,1467792
8864,VTN01,I23-009855,19/12/2023,11010001,MLXHYDR01,Hydrocortisone cream 1% (L) (5,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400002,480002,230300,12REH,12REH,1PHI,IPD,AR,THB,51.38,10.28,707302.28,93231.0,DRMS01,93231,พีรวุฒิ ตันติสุวณิชย์กุล นพ.,300.0,13652444,34,8486182,1467792
8863,VTN01,I23-009855,19/12/2023,11010001,MLXHYDR01,Hydrocortisone cream 1% (L) (5,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400002,480002,230300,12REH,12REH,1PHI,IPD,AR,THB,31.40,6.28,707302.28,93231.0,DRMS01,93231,พีรวุฒิ ตันติสุวณิชย์กุล นพ.,300.0,13652444,34,8486182,1467792
8862,VTN01,I23-009855,19/12/2023,11010001,MLXHYDR01,Hydrocortisone cream 1% (L) (5,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400002,480002,230300,12REH,12REH,1PHI,IPD,AR,THB,2.85,0.57,707302.28,93231.0,DRMS01,93231,พีรวุฒิ ตันติสุวณิชย์กุล นพ.,300.0,13652444,34,8486182,1467792
9437,VTN01,I23-009855,19/12/2023,11010001,MLXHYDR01,Hydrocortisone cream 1% (L) (5,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400002,480002,230300,12REH,12REH,1PHI,IPD,AR,THB,51.38,10.28,1881.69,93231.0,DRMS01,93231,พีรวุฒิ ตันติสุวณิชย์กุล นพ.,300.0,13652444,34,8486182,1467792
9465,VTN01,I23-009855,19/12/2023,11010001,MOTERDO01,Erdos cap 300 mg [ND],NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400001,480001,230300,12REH,2MED,1PHI,IPD,AR,THB,0.00,0.00,0.02,93306.0,DRMS01,93306,ธนกร ทรรศนียศิลป์ นพ.,816.0,13652444,95,8486182,1467792
8878,VTN01,I23-009855,19/12/2023,11010001,MOTERDO01,Erdos cap 300 mg [ND],NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400001,480001,230300,12REH,2MED,1PHI,IPD,AR,THB,1656.00,331.20,707302.28,93306.0,DRMS01,93306,ธนกร ทรรศนียศิลป์ นพ.,816.0,13652444,95,8486182,1467792
8865,VTN01,I23-009855,26/12/2023,11010001,MOTBIOQ02,BIO-QUINONE Q10 cap 100 mg (R),NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400001,480001,230300,12REH,2VNC,1PHI,IPD,AR,THB,1224.00,244.80,707302.28,91857.0,DRMS01,91857,นริศ สมิตาสิน นพ.,265.0,13652444,431,8486182,1467792
9458,VTN01,I23-009855,26/12/2023,11010001,MOTBIOQ02,BIO-QUINONE Q10 cap 100 mg (R),NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400001,480001,230300,12REH,2VNC,1PHI,IPD,AR,THB,0.00,0.00,0.02,91857.0,DRMS01,91857,นริศ สมิตาสิน นพ.,265.0,13652444,431,8486182,1467792
8870,VTN01,I23-009855,27/12/2023,11010001,MONBERO01,Berodual Forte UDV (R)nebule(4,NaN,23-029116,Mr Zayed Khalfan Abdulla Saeed,400001,480001,230300,12REH,2MED,1PHI,IPD,AR,THB,7105.00,1421.00,707302.28,93306.0,DRMS01,93306,ธนกร ทรรศนียศิลป์ นพ.,3168.0,13652444,540,8486182,1467792


In [37]:
onw20240912[[
"OrderDate",
"Package Code", 
"Account Code" ,
'OrderItemID and Accession No',
'OrderItemSubID and Accession No',
    'Episode Id',
    'Patient Id'
]].value_counts()


OrderDate   Package Code  Account Code  OrderItemID and Accession No  OrderItemSubID and Accession No  Episode Id  Patient Id
25/08/2024  7AB2          455001        15756761                      527                              8768743     1497614       2
11/09/2024  IP10B1        455001        16028835                      81                               8804726     1323912       2
                          441002        16028835                      80                               8804726     1323912       2
                                        16034844                      69                               8805685     1488314       2
                                        16034946                      104                              8805694     1418431       2
                                                                                                                                ..
08/09/2024  VCC0000002    410001        15938381                      630               

In [34]:
filtered_df = onw20240912[
    (onw20240912['OrderItemID and Accession No'] == 16028835) &
    (onw20240912['OrderItemSubID and Accession No'] == 81)
]

In [35]:
filtered_df

,BU,en,OrderDate,Billing Sub Group,Item code,Description,Package Code,hn,Name,Account Code,Discoun Account,Department,First Loc,Patient Loc,Rec Loc,Sub Type,nation,curr,Amount Before,Discount,Bill Amount,Doctor Group,Doctor Type,Doctor Code,Doctor Name,Qty,OrderItemID and Accession No,OrderItemSubID and Accession No,Episode Id,Patient Id
22302,VTN01,I24-007259,11/09/2024,11120000,NUIPDNURS101,ค่าการพยาบาลห้อง Grand Single,IP10B1,19-024907,ด.ญ. มิลิน วรสิทธิกร,455001,485501,110100,2KID,2W9A,2W9A,IPD,TH,THB,300.0,0.0,9700.0,NaN,NaN,NaN,NaN,1.0,16028835,81,8804726,1323912
22376,VTN01,I24-007259,11/09/2024,11120000,NUIPDNURS101,ค่าการพยาบาลห้อง Grand Single,IP10B1,19-024907,ด.ญ. มิลิน วรสิทธิกร,455001,485501,110100,2KID,2W9A,2W9A,IPD,TH,THB,2800.0,0.0,53166.0,NaN,NaN,NaN,NaN,1.0,16028835,81,8804726,1323912


In [ ]:

find_unique_combinations(onw20240912, [
'OrderDate','OrderItemID and Accession No','OrderItemSubID and Accession No'
])